In [3]:
## Importing needed packages and functions for model

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import copy
from PIL import Image
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
import torch.optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torchsummary import summary
import neptune

pl.seed_everything(2023)


/Users/adamwalker/opt/anaconda3/envs/Deep_Learning/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
## Importing packages for visualization

from misc_functions import get_example_params, save_class_activation_images, preprocess_image
from PIL import Image


In [4]:
## Loading data
labels_df = pd.read_csv('train_labels.csv')

In [6]:
## Adding the same model architecture

## Creating the model

class HistopathClassifier(pl.LightningModule):
    def __init__(self):
        super(HistopathClassifier, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 12 * 12, 512)
        self.fc2 = nn.Linear(512, 1)
        self.dropout = nn.Dropout(0.25)
        self.sigmoid = nn.Sigmoid()
        self.loss = nn.BCELoss()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 12 * 12)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return self.sigmoid(x)
    
    def train_step(self, inputs, labels, criterion, optimizer):
        optimizer.zero_grad()
        outputs = self(inputs).squeeze()
        
        if outputs.squeeze().dim() == 0: #done for rare instances where a batch could have only 1 sample
            outputs = outputs.unsqueeze(0)
    
        #print(outputs.shape)
        #print(outputs.squeeze().shape)
        #print(labels.float().shape)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        return loss.item()

    def validate_step(self, inputs, labels, criterion):
        with torch.no_grad():
            outputs = self(inputs).squeeze()
            if outputs.squeeze().dim() == 0: #done for rare instances where a batch could have only 1 sample
                outputs = outputs.unsqueeze(0)
            
            loss = criterion(outputs, labels.float())
        return loss.item()



In [7]:
## Double checking it looks the same

## Creating the model and getting a summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


num_epochs = 20
model = HistopathClassifier()
model.to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)


summary(model, input_size=(3, 96, 96),device=device.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 96, 96]             448
         MaxPool2d-2           [-1, 16, 48, 48]               0
            Conv2d-3           [-1, 32, 48, 48]           4,640
         MaxPool2d-4           [-1, 32, 24, 24]               0
            Conv2d-5           [-1, 64, 24, 24]          18,496
         MaxPool2d-6           [-1, 64, 12, 12]               0
           Dropout-7                 [-1, 9216]               0
            Linear-8                  [-1, 512]       4,719,104
           Dropout-9                  [-1, 512]               0
           Linear-10                    [-1, 1]             513
          Sigmoid-11                    [-1, 1]               0
Total params: 4,743,201
Trainable params: 4,743,201
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.11
Forw

In [11]:
## Loading from weights trained on the main script

model.load_state_dict(torch.load("my_model.pt",map_location=torch.device('cpu')))


<All keys matched successfully>

In [26]:
from torchvision import models

pretrained_model = models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /Users/adamwalker/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth
100%|██████████| 233M/233M [00:19<00:00, 12.5MB/s] 


In [37]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),

            nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),

            nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
        )
        self.fc_layers = nn.Sequential(
            nn.Dropout(p=0.25, inplace=False),
            nn.Linear(in_features=64 * 12 * 12, out_features=512, bias=True),
            nn.ReLU(),

            nn.Dropout(p=0.25, inplace=False),
            nn.Linear(in_features=512, out_features=1, bias=True),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 64 * 12 * 12)
        x = self.fc_layers(x)
        return x

fake_model = MyModel()
print(fake_model._modules['conv_layers'])
#print(fake_model.fc_layers)

Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU()
  (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [30]:

model._modules


OrderedDict([('conv1',
              Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
             ('conv2',
              Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
             ('conv3',
              Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
             ('pool',
              MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)),
             ('fc1', Linear(in_features=9216, out_features=512, bias=True)),
             ('fc2', Linear(in_features=512, out_features=1, bias=True)),
             ('dropout', Dropout(p=0.25, inplace=False)),
             ('sigmoid', Sigmoid()),
             ('loss', BCELoss())])

In [31]:
pretrained_model._modules['features']
pretrained_model._modules['classifier']

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [39]:
## Attempting LRP on a single iamge 
from LRP import LRP

input_images_list = os.listdir("input_images")

img = input_images_list[0]
path_to_image = os.path.join("input_images", img)

original_image = Image.open(path_to_image).convert('RGB')
prep_img = preprocess_image(original_image)

layerwise_relevance = LRP(fake_model)

LRP_per_layer = layerwise_relevance.generate(prep_img, 0)


KeyError: 'features'